In [1]:
import pyarrow as pa
import pyarrow.flight

# client = pa.flight.connect("grpc://0.0.0.0:8815")
client = pa.flight.connect(location='grpc://192.168.24.90:8816')

# Upload a new dataset
NUM_BATCHES = 1024
ROWS_PER_BATCH = 4096
upload_descriptor = pa.flight.FlightDescriptor.for_path("streamed.parquet")
batch = pa.record_batch([
    pa.array(range(ROWS_PER_BATCH)),
], names=["ints"])
writer, _ = client.do_put(upload_descriptor, batch.schema)
with writer:
    for _ in range(NUM_BATCHES):
        writer.write_batch(batch)

In [7]:
for i in client.list_flights():
    print(i)

<pyarrow.flight.FlightInfo schema=ints: int64 descriptor=<pyarrow.flight.FlightDescriptor path=[b'streamed.parquet']> endpoints=[<pyarrow.flight.FlightEndpoint ticket=<pyarrow.flight.Ticket ticket=b'streamed.parquet'> locations=[<pyarrow.flight.Location b'grpc://0.0.0.0:8816'>]>] total_records=4194304 total_bytes=796>


In [6]:
# Read content of the dataset
flight = client.get_flight_info(upload_descriptor)
reader = client.do_get(flight.endpoints[0].ticket)
total_rows = 0
for chunk in reader:
    total_rows += chunk.data.num_rows
print("Got", total_rows, "rows total, expected", NUM_BATCHES * ROWS_PER_BATCH)

Got 4194304 rows total, expected 4194304
